# 🏋️‍♀️ Health & Fitness Evaluations with Azure AI Foundry 🏋️‍♂️

This notebook demonstrates how to **evaluate** a Generative AI model (or application) using the **Azure AI Foundry** ecosystem. We'll highlight three key Python SDKs:
1. **`azure-ai-projects`** (`AIProjectClient`): manage & orchestrate evaluations in the cloud.
2. **`azure-ai-inference`**: perform model inference (optional but helpful if generating data for evaluation).
3. **`azure-ai-evaluation`**: run automated metrics for LLM output quality & safety.

We'll create or use some synthetic "health & fitness" Q&A data, then measure how well your model is answering. We'll do both **local** evaluation and **cloud** evaluation (on an Azure AI Foundry project).

> **Disclaimer**: This covers a hypothetical health & fitness scenario. **No real medical advice** is provided. Always consult professionals.

## Notebook Contents
1. [Setup & Imports](#1-Setup-and-Imports)
2. [Local Evaluation Examples](#3-Local-Evaluation)
3. [Cloud Evaluation with `AIProjectClient`](#4-Cloud-Evaluation)
4. [Extra Topics](#5-Extra-Topics)
   - [Risk & Safety Evaluators](#5.1-Risk-and-Safety)
   - [More Quality Evaluators](#5.2-Quality)
   - [Custom Evaluators](#5.3-Custom)
   - [Simulators & Adversarial Data](#5.4-Simulators)
5. [Conclusion](#6-Conclusion)


## 1. Setup and Imports
We'll install necessary libraries, import them, and define some synthetic data. 

### Dependencies
- `azure-ai-projects` for orchestrating evaluations in your Azure AI Foundry Project.
- `azure-ai-evaluation` for built-in or custom metrics (like Relevance, Groundedness, F1Score, etc.).
- `azure-ai-inference` (optional) if you'd like to generate completions to produce data to evaluate.
- `azure-identity` (for Azure authentication via `DefaultAzureCredential`).

### Synthetic Data
We'll create a small JSONL with *health & fitness* Q&A pairs, including `query`, `response`, `context`, and `ground_truth`. This simulates a scenario where we have user questions, the model's answers, plus a reference ground truth.

You can adapt this approach to any domain: e.g., finance, e-commerce, etc.

<img src="./seq-diagrams/2-evals.png" alt="Evaluation Flow" width="30%"/>


In [1]:
# Import required Azure libraries
import os
import json  # For JSON operations
from pathlib import Path


def find_cred_json(start_path):
    # Start from current directory and go up
    current = Path(start_path)
    while current != current.parent:  # while we haven't hit the root
        cred_file = current / 'cred.json'
        if cred_file.exists():
            return str(cred_file)
        current = current.parent
    return None

try:
    # Search in the parent directory and its subdirectories
    parent_dir = os.path.dirname(os.getcwd())  # Get parent directory
    file_path = find_cred_json(parent_dir)

    if not file_path:
        raise FileNotFoundError("cred.json not found in parent directories")

    print(f"Found cred.json at: {file_path}")

    # Load and parse the JSON file
    with open(file_path, 'r') as f:
        loaded_config = json.load(f)

    
except Exception as e:
    print(f"❌ Error creating search clients: {e}")

Found cred.json at: D:\MLOps\Gen Ai & MLOps Masterclass\Materilas\test\ai-foundry-workshop\cred.json


In [2]:
%%capture
# If you need to install these, uncomment:
# !pip install azure-ai-projects azure-ai-evaluation azure-ai-inference azure-identity
# !pip install opentelemetry-sdk azure-core-tracing-opentelemetry  # optional for advanced tracing

import json
import os
import uuid
from pathlib import Path
from typing import Dict, Any

from azure.identity import DefaultAzureCredential

# We'll create a synthetic dataset in JSON Lines format
synthetic_eval_data = [
    {
        "query": "How can I start a beginner workout routine at home?",
        "context": "Workout routines can include push-ups, bodyweight squats, lunges, and planks.",
        "response": "You can just go for 10 push-ups total.",
        "ground_truth": "At home, you can start with short, low-intensity workouts: push-ups, lunges, planks."
    },
    {
        "query": "Are diet sodas healthy for daily consumption?",
        "context": "Sugar-free or diet drinks may reduce sugar intake, but they still contain artificial sweeteners.",
        "response": "Yes, diet sodas are 100% healthy.",
        "ground_truth": "Diet sodas have fewer sugars than regular soda, but 'healthy' is not guaranteed due to artificial additives."
    },
    {
        "query": "What's the capital of France?",
        "context": "France is in Europe. Paris is the capital.",
        "response": "London.",
        "ground_truth": "Paris."
    }
]

# Write them to a local JSONL file
eval_data_path = Path("./health_fitness_eval_data.jsonl")
with eval_data_path.open("w", encoding="utf-8") as f:
    for row in synthetic_eval_data:
        f.write(json.dumps(row) + "\n")

print(f"Sample evaluation data written to {eval_data_path.resolve()}")

# 3. Local Evaluation Examples

We'll show how to run local, code-based evaluation on a JSONL dataset. We'll:
1. **Load** the data.
2. **Define** one or more evaluators. (e.g. `F1ScoreEvaluator`, `RelevanceEvaluator`, `GroundednessEvaluator`, or custom.)
3. **Run** `evaluate(...)` to produce a dictionary of metrics.

> We can also do multi-turn conversation data or add extra columns like `ground_truth` for advanced metrics.

## Example 1: Combining F1Score, Relevance & Groundedness
We'll combine:
- `F1ScoreEvaluator` (NLP-based, compares `response` to `ground_truth`)
- `RelevanceEvaluator` (AI-assisted, uses GPT to judge how well `response` addresses `query`)
- `GroundednessEvaluator` (checks how well the response is anchored in the provided `context`)
- A custom code-based evaluator that logs response length.


In [3]:
import os
from azure.ai.evaluation import (
    evaluate,
    F1ScoreEvaluator,
    RelevanceEvaluator,
    GroundednessEvaluator
)

# Our custom evaluator to measure response length.
def response_length_eval(response, **kwargs):
    return {"resp_length": len(response)}

# We'll define an example GPT-based config (if we want AI-assisted evaluators). 
# This is needed for AI-assisted evaluators. Fill with your Azure OpenAI config.
# If you skip some evaluators, you can omit.
model_config = {
    "azure_endpoint": os.environ.get("AOAI_ENDPOINT", "https://dummy-endpoint.azure.com"),
    "api_key": os.environ.get("AOAI_API_KEY", "fake-key"),
    "azure_deployment": os.environ.get("AOAI_DEPLOYMENT", "gpt-4"),
    "api_version": os.environ.get("AOAI_API_VERSION", "2023-07-01-preview"),
}

f1_eval = F1ScoreEvaluator()
rel_eval = RelevanceEvaluator(model_config=model_config)
ground_eval = GroundednessEvaluator(model_config=model_config)

# We'll run evaluate(...) with these evaluators.
results = evaluate(
    data=str(eval_data_path),
    evaluators={
        "f1_score": f1_eval,
        "relevance": rel_eval,
        "groundedness": ground_eval,
        "resp_len": response_length_eval
    },
    evaluator_config={
        "f1_score": {
            "column_mapping": {
                "response": "${data.response}",
                "ground_truth": "${data.ground_truth}"
            }
        },
        "relevance": {
            "column_mapping": {
                "query": "${data.query}",
                "response": "${data.response}"
            }
        },
        "groundedness": {
            "column_mapping": {
                "context": "${data.context}",
                "response": "${data.response}"
            }
        },
        "resp_len": {
            "column_mapping": {
                "response": "${data.response}"
            }
        }
    }
)

print("Local evaluation result =>")
print(results)

[2025-02-28 18:59:35 +0530][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-02-28 18:59:35 +0530][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-02-28 18:59:35 +0530][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-02-28 18:59:35 +0530][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-02-28 18:59:35 +0530][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run main_response_length_eval_gdfuudmi_20250228_185933_779232, log path: C:\Users\sarat\.promptflow\.runs\main_respo

2025-02-28 18:59:35 +0530   25304 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-02-28 18:59:35 +0530   25304 execution.bulk     INFO     Finished 3 / 3 lines.
2025-02-28 18:59:35 +0530   25304 execution.bulk     INFO     Average execution time for completed lines: 0.01 seconds. Estimated time for incomplete lines: 0.0 seconds.
======= Run Summary =======

Run name: "azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_m7ebnne4_20250228_185933_778232"
Run status: "Completed"
Start time: "2025-02-28 18:59:33.773238+05:30"
Duration: "0:00:02.672717"
Output path: "C:\Users\sarat\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_m7ebnne4_20250228_185933_778232"

2025-02-28 18:59:37 +0530   25304 execution.bulk     INFO     Process name(SpawnProcess-2)-Process id(9672)-Line number(0) start execution.
2025-02-28 18:59:37 +0530   25304 execution.bulk     INFO     Process name

[2025-02-28 18:59:38 +0530][promptflow.core._prompty_utils][ERROR] - Exception occurs: APIConnectionError: Connection error.
[2025-02-28 18:59:38 +0530][promptflow.core._prompty_utils][ERROR] - Exception occurs: APIConnectionError: Connection error.
[2025-02-28 18:59:38 +0530][promptflow.core._prompty_utils][ERROR] - Exception occurs: APIConnectionError: Connection error.
[2025-02-28 18:59:38 +0530][promptflow.core._prompty_utils][ERROR] - Exception occurs: APIConnectionError: Connection error.
[2025-02-28 18:59:38 +0530][promptflow.core._prompty_utils][ERROR] - Exception occurs: APIConnectionError: Connection error.
[2025-02-28 18:59:38 +0530][promptflow.core._prompty_utils][ERROR] - Exception occurs: APIConnectionError: Connection error.


2025-02-28 18:59:38 +0530   25304 execution.bulk     INFO     Finished 3 / 3 lines.
2025-02-28 18:59:38 +0530   25304 execution.bulk     INFO     Finished 3 / 3 lines.
2025-02-28 18:59:38 +0530   25304 execution.bulk     INFO     Average execution time for completed lines: 0.88 seconds. Estimated time for incomplete lines: 0.0 seconds.
2025-02-28 18:59:38 +0530   25304 execution.bulk     INFO     Average execution time for completed lines: 0.89 seconds. Estimated time for incomplete lines: 0.0 seconds.
2025-02-28 18:59:38 +0530   25304 execution          ERROR    3/3 flow run failed, indexes: [2,0,1], exception of index 2: OpenAI API hits APIConnectionError: Connection error. [Error reference: https://platform.openai.com/docs/guides/error-codes/api-errors]
2025-02-28 18:59:38 +0530   25304 execution          ERROR    3/3 flow run failed, indexes: [0,2,1], exception of index 0: OpenAI API hits APIConnectionError: Connection error. [Error reference: https://platform.openai.com/docs/guide

[2025-02-28 18:59:38 +0530][promptflow._sdk._orchestrator.run_submitter][WARNING] - 3 out of 3 runs failed in batch run.
 Please check out C:/Users/sarat/.promptflow/.runs/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_ys_r2wr8_20250228_185933_781227 for more details.
[2025-02-28 18:59:38 +0530][promptflow._sdk._orchestrator.run_submitter][WARNING] - 3 out of 3 runs failed in batch run.
 Please check out C:/Users/sarat/.promptflow/.runs/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_7cu2aqvg_20250228_185933_779232 for more details.


...

[2025-02-28 18:59:47 +0530][promptflow._sdk._orchestrator.run_submitter][WARNING] - 3 out of 3 runs failed in batch run.
 Please check out C:/Users/sarat/.promptflow/.runs/main_response_length_eval_gdfuudmi_20250228_185933_779232 for more details.


2025-02-28 18:59:35 +0530   25304 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-02-28 18:59:35 +0530   25304 execution.bulk     INFO     The timeout for the batch run is 3600 seconds.
2025-02-28 18:59:35 +0530   25304 execution.bulk     INFO     Current system's available memory is 15434.73828125MB, memory consumption of current process is 234.30078125MB, estimated available worker count is 15434.73828125/234.30078125 = 65
2025-02-28 18:59:35 +0530   25304 execution.bulk     INFO     Set process count to 3 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 3, 'estimated_worker_count_based_on_memory_usage': 65}.
2025-02-28 18:59:37 +0530   25304 execution.bulk     INFO     Process name(SpawnProcess-2)-Process id(9672)-Line number(0) start execution.
2025-02-28 18:59:37 +0530   25304 execution.bulk     INFO     Process name(SpawnProcess-3)-Process id(7072)-Line number(1) start ex

D:\MLOps\Gen Ai & MLOps Masterclass\Materilas\test\ai-foundry-workshop\.venv\Lib\site-packages\azure\ai\ml\_restclient\v2023_10_01\models\_azure_machine_learning_services_enums.py:208: SyntaxWarning: invalid escape sequence '\ '
  """Random forest is a supervised learning algorithm.
D:\MLOps\Gen Ai & MLOps Masterclass\Materilas\test\ai-foundry-workshop\.venv\Lib\site-packages\azure\ai\ml\_restclient\v2023_10_01\models\_azure_machine_learning_services_enums.py:1327: SyntaxWarning: invalid escape sequence '\ '
  """Random forest is a supervised learning algorithm.
D:\MLOps\Gen Ai & MLOps Masterclass\Materilas\test\ai-foundry-workshop\.venv\Lib\site-packages\azure\ai\ml\_vendor\azure_resources\operations\_resources_operations.py:65: SyntaxWarning: invalid escape sequence '\ '
  """Get all the resources for a resource group.
D:\MLOps\Gen Ai & MLOps Masterclass\Materilas\test\ai-foundry-workshop\.venv\Lib\site-packages\azure\ai\ml\_vendor\azure_resources\operations\_resources_operations.py:

✅ Uploaded JSONL to project. Data asset ID: /subscriptions/1c2fd79b-ad21-4ad0-8d53-12de16650452/resourceGroups/rg-sarath-1834_ai/providers/Microsoft.MachineLearningServices/workspaces/sarath-1178/data/7c8292b2-339f-4b2d-9c9c-7d84d284921d/versions/1


✅ Created AIProjectClient.
✅ Uploaded JSONL to project. Data asset ID: /subscriptions/1c2fd79b-ad21-4ad0-8d53-12de16650452/resourceGroups/rg-sarath-1834_ai/providers/Microsoft.MachineLearningServices/workspaces/sarath-1178/data/d48923a1-816b-4cd4-883d-5596c4af9fc4/versions/1


**Inspecting Local Results**

The `evaluate(...)` call returns a dictionary with:
- **`metrics`**: aggregated metrics across rows (like average F1, Relevance, or Groundedness)
- **`rows`**: row-by-row results with inputs and evaluator outputs
- **`traces`**: debugging info (if any)

You can further analyze these results, store them in a database, or integrate them into your CI/CD pipeline.

# 4. Cloud Evaluation with `AIProjectClient`

Sometimes, we want to:
- Evaluate large or sensitive datasets in the cloud (scalability, governed access).
- Keep track of evaluation results in an Azure AI Foundry project.
- Optionally schedule recurring evaluations.

We'll do that by:
1. **Upload** the local JSONL to your Azure AI Foundry project.
2. **Create** an `Evaluation` referencing built-in or custom evaluator definitions.
3. **Poll** until the job is done (with retry logic for resilience).
4. **Review** the results in the portal or via `project_client.evaluations.get(...)`.

### Prerequisites
- An Azure AI Foundry project with a valid **Connection String** (from your project’s Overview page).
- An Azure OpenAI deployment (if using AI-assisted evaluators).


In [8]:
!pip install azure-ai-ml azure-identity azure-core azure-ai-projects azure-ai-evaluation

  Using cached azure_ai_evaluation-1.3.0-py3-none-any.whl.metadata (32 kB)
  Using cached promptflow_devkit-1.17.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached promptflow_core-1.17.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl.metadata (3.0 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached promptflow_tracing-1.17.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached flask_cors-5.0.1-py3-none-any.whl.metadata (961 bytes)
  Using cached flask_restx-1.3.0-py2.py3-none-any.whl.metadata (9.3 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.30.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached pydash-7.0.7-py3-none-any.whl.metadata (45 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.30.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached opentelemetry_pro

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.3 which is incompatible.


In [4]:
!pip list

Package                                  Version
---------------------------------------- ------------------
aiobotocore                              2.12.3
aiohttp                                  3.9.5
aioitertools                             0.7.1
aiosignal                                1.2.0
alabaster                                0.7.16
altair                                   5.0.1
anaconda-anon-usage                      0.4.4
anaconda-catalogs                        0.2.0
anaconda-client                          1.12.3
anaconda-cloud-auth                      0.5.1
anaconda-navigator                       2.6.0
anaconda-project                         0.11.1
aniso8601                                10.0.0
annotated-types                          0.6.0
anyio                                    4.2.0
appdirs                                  1.4.4
archspec                                 0.2.3
argcomplete                              3.5.3
argon2-cffi                             

In [6]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation, Dataset, EvaluatorConfiguration, ConnectionType
)
from azure.ai.evaluation import F1ScoreEvaluator, RelevanceEvaluator, ViolenceEvaluator
from azure.identity import DefaultAzureCredential
from azure.core.exceptions import ServiceResponseError
import time

# 1) Connect to Azure AI Foundry project
project_conn_str = loaded_config.get("PROJECT_CONNECTION_STRING")
credential = DefaultAzureCredential()

project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=project_conn_str
)
print("✅ Created AIProjectClient.")

# 2) Upload data for evaluation
uploaded_data_id, _ = project_client.upload_file(str(eval_data_path))
print("✅ Uploaded JSONL to project. Data asset ID:", uploaded_data_id)

# 3) Prepare an Azure OpenAI connection for AI-assisted evaluators
default_conn = project_client.connections.get_default(ConnectionType.AZURE_OPEN_AI)

deployment_name = os.environ.get("AOAI_DEPLOYMENT", "gpt-4")
api_version = os.environ.get("AOAI_API_VERSION", "2023-07-01-preview")

# 4) Construct the evaluation object
model_config = default_conn.to_evaluator_model_config(
    deployment_name=deployment_name,
    api_version=api_version
)

evaluation = Evaluation(
    display_name="Health Fitness Remote Evaluation",
    description="Evaluating dataset for correctness.",
    data=Dataset(id=uploaded_data_id),
    evaluators={
        "f1_score": EvaluatorConfiguration(id=F1ScoreEvaluator.id),
        "relevance": EvaluatorConfiguration(
            id=RelevanceEvaluator.id,
            init_params={"model_config": model_config}
        ),
        "violence": EvaluatorConfiguration(
            id=ViolenceEvaluator.id,
            init_params={"azure_ai_project": project_client.scope}
        )
    }
)

# Helper: Create evaluation with retry logic
def create_evaluation_with_retry(project_client, evaluation, max_retries=3, retry_delay=5):
    for attempt in range(max_retries):
        try:
            result = project_client.evaluations.create(evaluation=evaluation)
            return result
        except ServiceResponseError as e:
            if attempt == max_retries - 1:
                raise
            print(f"⚠️ Attempt {attempt+1} failed: {str(e)}. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)

# 5) Create & track the evaluation using retry logic
cloud_eval = create_evaluation_with_retry(project_client, evaluation)
print("✅ Created evaluation job. ID:", cloud_eval.id)

# 6) Poll or fetch final status
fetched_eval = project_client.evaluations.get(cloud_eval.id)
print("Current status:", fetched_eval.status)
if hasattr(fetched_eval, 'properties'):
    link = fetched_eval.properties.get("AiStudioEvaluationUri", "")
    if link:
        print("View details in Foundry:", link)
else:
    print("No link found.")

TypeError: ConnectionsOperations.get_default() takes 1 positional argument but 2 were given

### Viewing Cloud Evaluation Results
- Navigate to the **Evaluations** tab in your AI Foundry project to see your evaluation job.
- Open the evaluation to view aggregated metrics and row-level details.
- For AI-assisted or risk & safety evaluators, you'll see both average scores and detailed per-row results.

# 5. Extra Topics
We'll do a quick overview of some advanced features:
1. [Risk & Safety Evaluators](#5.1-Risk-and-Safety)
2. [Additional Quality Evaluators](#5.2-Quality)
3. [Custom Evaluators](#5.3-Custom)
4. [Simulators & Adversarial Data](#5.4-Simulators)


## 5.1 Risk & Safety Evaluators

Azure AI Foundry includes built-in evaluators that detect content risks. Examples include:
- **ViolenceEvaluator**: detects violent or harmful content.
- **SexualEvaluator**: checks for explicit content.
- **HateUnfairnessEvaluator**: flags hateful content.
- **SelfHarmEvaluator**: detects self-harm related content.
- **ProtectedMaterialEvaluator**: identifies copyrighted or protected content.

These evaluators accept a `query` and `response` (and sometimes `context`) to provide severity labels and scores.

For example:
```python
from azure.ai.evaluation import ViolenceEvaluator

violence_eval = ViolenceEvaluator(
    credential=DefaultAzureCredential(),
    azure_ai_project={
        "subscription_id": "...",
        "resource_group_name": "...",
        "project_name": "..."
    }
)
result = violence_eval(query="What is the capital of France?", response="Paris")
print(result)
```


## 5.2 Additional Quality Evaluators
Beyond `F1Score` and `Relevance`, there are many built-ins:
- **GroundednessEvaluator**: Checks if the response is anchored in the provided context.
- **CoherenceEvaluator**: Measures the logical flow of the response.
- **FluencyEvaluator**: Assesses grammatical correctness.

These metrics can help you fine-tune your model’s performance.

## 5.3 Custom Evaluators
You can build your own evaluators. For instance, a simple evaluator that measures the length of a response:
```python
class AnswerLengthEvaluator:
    def __call__(self, response: str, **kwargs):
        return {"answer_length": len(response)}
```

You can then integrate it with the local or cloud evaluation workflow.

## 5.4 Simulators & Adversarial Data
If you need to generate synthetic or adversarial evaluation data, the `azure-ai-evaluation` package provides simulators. 

For example, you can simulate adversarial queries using `AdversarialSimulator` to test model safety and robustness.

# 6. Conclusion 🏁

We covered:
1. **Local** evaluations with `evaluate(...)` on JSONL data (now including a groundedness metric).
2. **Cloud** evaluations with `AIProjectClient` including retry logic for robustness.
3. Built-in **risk & safety** and **quality** evaluators.
4. **Custom** evaluators for advanced scenarios.
5. **Simulators** for generating adversarial data.

**Next Steps**:
- Adjust your model and prompts based on evaluation feedback.
- Integrate these evaluations into your CI/CD pipelines.
- Combine with observability tools for deeper insights.

> **Best of luck** building robust and responsible AI solutions with Azure AI Foundry!